# Анализ данных на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Примеры работы с API ВКонтакте

## Подготовка к работе

### Знакомимся с документацией API

Для начала давайте посмотрим на [документацию](https://dev.vk.com/ru/api/api-requests) API ВКонтакте и посмотрим, как формировать запросы для получения информации. 

Общий вид запроса следующий:

    https://api.vk.com/method/<METHOD>?<PARAMS>
    
Вместо `<METHOD>` указывается название метода, вместо `<PARAMS>` – параметры этого метода в виде пар `<параметр>=<значение>`, объединённых через `&`. Какие-то параметры являются необязательными, какие-то – обязательными. Обязательными для сбора данных являются два параметра:

* токен доступа `access_token`;
* версия API `v`.

Примеры методов и параметров (вместо токена доступа число 1):

<table>
    <tr>
        <th>Запрос</th><th>Ссылка</th>
    </tr>
    <tr>
        <td>базовая информация по пользователю с id=743784474</td>
        <td>https://api.vk.com/method/users.get?user_ids=743784474&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>дата рождения пользователя с id=743784474</th>
        <td>https://api.vk.com/method/users.get?user_ids=743784474&fields=bdate&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>базовая информация по пользователям с id=743784474 и id=20473269</td>
        <td>https://api.vk.com/method/users.get?user_ids=743784474,20473269&fields=bdate&access_token=1&v=5.199</td>
    </tr>
    <tr>
        <td>100 последних постов со стены сообщества с названием hse</td>
        <td>https://api.vk.com/method/wall.get?domain=hse&count=100&access_token=1&v=5.199</td>
        </tr>
</table>

Итак, глобальная задача – получить токен доступа, сформировать ссылку для нужного запроса, отправить запрос к серверу и обработать полученный результат. Посмотрим, как отправляются запросы к серверу в Python.

### Модуль `requests` для отправки запросов

Базовый модуль `requests` в Python служит для отправки запросов типа `GET` или `POST`. В нашем случае достаточно запроса `GET` для получения информации, так как отправлять информацию на сервер нам не нужно (мы не пишем бота и не планируем удалённо управлять своим аккаунтом). Импортируем модуль, вызовем оттуда функцию `get()` и отправим какой-нибудь запрос из примеров выше:

In [1]:
import requests

In [2]:
req = requests.get("https://api.vk.com/method/users.get?user_ids=743784474&access_token=1&v=5.199")
print(req)

<Response [200]>


В переменной `req` хранится объект типа `Response`, он скрыт, временно хранится в некоторой ячейке памяти. Код 200 означает, что запрос отправлен и ответ от сервера получен, это стандартный код [состояния HTTP](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP). Этот код можно извлечь отдельно, вызвав атрибут `status_code`, это может пригодится для последующей работы (если код равен 200, продолжаем работу, если нет, реализуем программу по поиску ошибки и её причины):

In [3]:
# код типа integer
print(req.status_code, type(req.status_code))

200 <class 'int'>


Из `req` можем извлечь ссылку, ту, что в адресной строке (сейчас не так интересно, потому что мы ссылку сами отправили, но может пригодится):

In [4]:
print(req.url)

https://api.vk.com/method/users.get?user_ids=743784474&access_token=1&v=5.199


А можем сам результат в виде строки, в данном случае это текст, который мы видим на странице при переходе по ссылке выше в браузере:

In [5]:
# строка, тип str
print(req.text, type(req.text))

{"error":{"error_code":5,"error_msg":"User authorization failed: invalid access_token (4).","request_params":[{"key":"user_ids","value":"743784474"},{"key":"v","value":"5.199"},{"key":"method","value":"users.get"},{"key":"oauth","value":"1"}]}} <class 'str'>


Готового результата с информацией по пользователю с указанным id мы не получили, поскольку токен доступа был указан некорректный. Однако видно, что, в любом случае, результат извлекается, он непустой и представлен в виде JSON-строки. Такую строку можно обработать с помощью модуля `json`, в нём есть функция `loads()` для десериализации JSON-строки и превращения её в питоновский словарь. Но мы поступим проще – применим метод `.json()`, который определён на объектах класса `Response`:

In [6]:
# словарь, тип dict
print(req.json(), type(req.json()))

{'error': {'error_code': 5, 'error_msg': 'User authorization failed: invalid access_token (4).', 'request_params': [{'key': 'user_ids', 'value': '743784474'}, {'key': 'v', 'value': '5.199'}, {'key': 'method', 'value': 'users.get'}, {'key': 'oauth', 'value': '1'}]}} <class 'dict'>


В результате мы получили готовый словарь, из которого можно извлекать необходимую информацию по ключам! Однако для того, чтобы информация в словаре была, нужно получить токен доступа для работы.

### Получаем токен доступа

Итак, алгоритм получения токена доступа:

1. Создаём новое приложение в разделе *Мои приложения* (https://vk.com/apps?act=manage) типа *Standalone-приложение* (поле *Платформа*).

2. Переходим в предлагаемый сервис для авторизации, при первой работе с сервисом вписываем свои данные (относительно новая политика безопасности vk), выбираем *Add app*, вписываем название приложения, в качестве платформы выбираем *Web*, в качестве домена и ссылки можно вписать предлагаемые для примера `mysite.com` и `https://mysite.com`, так как само приложение, связанное с каким-то своим сервером нам не нужно, нужен только ключ для последующего сбора данных.

3. Когда приложение создано, копируем его числовой ID (*App ID* из *App information*) и сохраняем. 

4. Для получения токена доступа формируем ссылку вида:

        https://oauth.vk.com/authorize?client_id=i&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token

и вместо `i` в `client_id` подставляем туда ID приложения. Переходим по ссылке, из адресной строки копируем токен доступа – набор символов после `access_token` и до `&expires_in`. Никому не показываем токен, так как он даёт доступ к вашему аккаунту. Токен действителен в течение суток при работе с того же IP адреса.

Для более удобного автоматизированного получения токена написан код ниже:

In [7]:
from getpass import getpass

In [8]:
# getpass() – аналог input(), скрывает вводимый текст
# подставляем id приложения и переходим по ссылке

app_id = getpass("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: ········
https://oauth.vk.com/authorize?client_id=51660535&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


In [9]:
# копируем ссылку из адресной строки (длинная с access_token)
# разбиваем ссылку на части и извлекаем токен -> token

full_link = getpass()
token = full_link.split("access_token=")[1].split("&")[0]

········


Теперь приступаем к работе!

## Выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества скалодрома [Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и короткое название сообщества:

In [10]:
v = "5.199"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Чтобы подставить необходимые параметры метода `wall.get`, а также токен доступа, можно воспользоваться обычным форматированием строк или f-строками, однако есть ещё более удобный способ. Функция `get()` из `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [11]:
# 100 – максимальное число постов за раз

params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

Теперь сформируем запрос:

In [12]:
req_wall = requests.get(main_wall, params = params_wall)
#print(req_wall.url)

Извлечём результаты и преобразуем JSON-строку в словарь:

In [13]:
json_wall = req_wall.json()

In [ ]:
#json_wall

### Задача 1

Извлеките из `json_wall` общее число постов на стене и сохраните его в переменную `nposts`. Извлеките из `json_wall` список словарей с информацией о каждом извлечённом после и сохраните его в переменную `items_wall`.

In [14]:
nposts = json_wall['response']['count']
items_wall = json_wall['response']['items']

### Задача 2

Выберите первый элемент списка `items_wall`, назовите его `i`. Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число просмотров;
* число комментариев.

In [15]:
i = items_wall[0]
print(i["id"], i["date"], i["text"])
print(i["likes"]["count"], i["views"]["count"], i["comments"]["count"])

4549 1715255770 Для простоты понимания и планирования тренировки можно разбить на 3 типа: 

1️⃣ Развивающие тренировки 

Цель этих тренировок - использовать принцип перегрузки, то есть применить нагрузку, величина которой превышает привычный для организма уровень. Можно лазить более тяжелые трассы или выполнять более тяжелые упражнения, то есть повысить интенсивность. Или же повысить объем - выполнить большее число подходов, увеличить длительность тренировки или количество трасс. Повышать и интенсивность и объем, как правило, не имеет смысла, так как может превысить наши адаптационные возможности. 

Без развивающих тренировок прогресс не возможен, но нужно понимать, что их количество не должно быть слишком большим. 

2️⃣ Закрепляющие тренировки 

Это тренировки со средней нагрузкой, которые не заставляют вас слишком сильно напрягаться. Вы не лазаете предельные трассы, вы не выполняете слишком тяжелые упражнения. Вместо того, чтобы добить «тот последний боулдеринг» вы берете себя в руки

### Задача 3

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все.

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [16]:
iterate = nposts // 100 + 1

### Задача 4

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [17]:
params_wall_long = {"access_token" : token, 
                    "domain" : domain, 
                    "count" : 100,
                    "v" : v,
                    "offset" : 0}

In [18]:
import time

In [19]:
# функция sleep() из time выставляет задержку запуска кода
# здесь задержка в 1.2 секунды

items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    items_all.extend(items_wall_long)
    params_wall_long["offset"] += 100
    
    time.sleep(1.2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


Проверьте длину списка `items_all` – все ли посты собраны. Преобразуйте полученный результат – список словарей – в датафрейм Pandas.

In [21]:
print(len(items_all))

1811


In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(items_all)
df.head()

,inner_type,donut,comments,marked_as_ads,short_text_rate,hash,has_translation,type,attachments,date,...,post_type,reposts,text,views,carousel_offset,edited,copy_history,geo,signer_id,zoom_text
0,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,0.8,g8vMeRCfINQYZ3gtFdoVX4lUkOg,False,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1715255770,...,post,"{'count': 5, 'user_reposted': 0}",Для простоты понимания и планирования трениров...,{'count': 833},NaN,NaN,NaN,NaN,NaN,NaN
1,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,0.8,OQLYU4cM2mRMq-_af4HqAvp6Q9Q,False,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1715169113,...,post,"{'count': 0, 'user_reposted': 0}",У нас на скалодроме всегда хорошая погода! \n\...,{'count': 591},0.0,NaN,NaN,NaN,NaN,NaN
2,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,0.8,jVhAZfrfvchnQqZJahxY3lfpY_Q,False,post,"[{'type': 'video', 'video': {'response_type': ...",1715093829,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 505},NaN,NaN,NaN,NaN,NaN,NaN
3,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 0, 'gr...",0,0.8,y0v5AZFMzFQYjGsQUybx1hlgbYk,False,post,"[{'type': 'video', 'video': {'response_type': ...",1714912279,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 463},NaN,NaN,NaN,NaN,NaN,NaN
4,wall_wallpost,{'is_donut': False},"{'can_post': 1, 'can_view': 1, 'count': 1, 'gr...",0,0.8,jZDiPQ7nJobsottgn1aa5NR1DVA,False,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1714901561,...,post,"{'count': 4, 'user_reposted': 0}",Воспитание психологической стойкости помогает ...,{'count': 958},NaN,NaN,NaN,NaN,NaN,NaN


### Задача 5

Создайте на основе полученного датафрейма новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, проверьте, есть ли в датафрейме пропущенные значения, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы. 

In [24]:
small = df[["id", "date", "text", "likes", "views", "comments"]]

In [25]:
# считаем число пропусков по всем столбцам
# views – раньше просмотры не фиксировались

small.isna().sum()

id            0
date          0
text          0
likes         0
views       870
comments      0
dtype: int64

In [26]:
# выключаем предупреждение SettingWithCopyWarning от pandas
# код ниже корректный, но pandas предупреждает о работе с копиями

pd.set_option('chained_assignment', None)

In [27]:
small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])

# так как есть пропуски, извлекаем count только в словарях
small["nviews"] = small["views"].apply(lambda x: x["count"] if type(x) is dict else x)

In [28]:
# более умный вариант кода, чтобы не писать одинаковые lambda-функции

def get_count(x):
    if type(x) is dict:
        return x["count"]
    else:
        return x
    
small["nlikes"] = small["likes"].apply(get_count)
small["ncomments"] = small["comments"].apply(get_count)
small["nviews"] = small["views"].apply(get_count)

In [29]:
# заполняем пропуски нулями и меняем тип столбца на integer
# из-за наличия пропусков всегда тип float

small["nviews"] = small["nviews"].fillna(0).astype(int)

In [30]:
# удаляем старые столбцы

small.drop(columns = ["likes", "views", "comments"], inplace = True)
small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1811 entries, 0 to 1810
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1811 non-null   int64 
 1   date       1811 non-null   int64 
 2   text       1811 non-null   object
 3   nlikes     1811 non-null   int64 
 4   ncomments  1811 non-null   int64 
 5   nviews     1811 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 85.0+ KB


### Задача 6

Добавьте в `small` столбец `date_time` с датой-временем поста в формате `datetime`.

In [31]:
# unit = 's': переводим из метки, которая является числом секунд с 1 января 1970

small["date_time"] = pd.to_datetime(small['date'], unit = 's')

### Задача 7

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [32]:
with_comm = small[small["ncomments"] > 0]

In [33]:
# число постов для оценки времени работы кода в бонусной части
print(with_comm.shape)

(601, 7)


## Выгружаем комментарии к постам (бонус)

Так как бонусная часть – время не экономим, возьмём не первые 10 постов, а все посты из `with_comm` и заберём id постов в виде списка:

In [34]:
ids = list(with_comm["id"])
print(ids[0:10]) # первые 10 id для примера

[4544, 4540, 4536, 4526, 4518, 4509, 4505, 4500, 4495, 4492]


Воспользуемся методом `wall.getComments`, который выгружает комментарии к посту по его id:

In [35]:
main_comm = "https://api.vk.com/method/wall.getComments"

Так как метод `wall.getComments` принимает на вход только один id за раз, нам нужно запустить цикл по списку `ids` (примерно 10 минут на код ниже, чуть большее 600 постов с задержкой в одну секунду):

In [36]:
# owner_id: id сообщества
# post_id: id поста
# v: версия API
# access_token: токен доступа
# count: число комментариев, 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [37]:
# пример комментария с ответами на них
print(comments_all[2])

{'id': 4537, 'from_id': 2210113, 'date': 1714247528, 'text': 'Снимите, пожалуйста, пролаз круга по пассивам с фиолетовыми метками 🙏', 'post_id': 4536, 'owner_id': -38936316, 'parents_stack': [], 'thread': {'count': 1, 'items': [{'id': 4539, 'from_id': 2847724, 'date': 1714325174, 'text': '[id2210113|Владимир], хорошо, постараемся в ближайшее время)', 'post_id': 4536, 'owner_id': -38936316, 'parents_stack': [4537], 'reply_to_user': 2210113, 'reply_to_comment': 4537}], 'can_post': True, 'show_reply_button': True, 'groups_can_post': True}}


In [38]:
df_comm = pd.DataFrame(comments_all)
print(df_comm.shape)
df_comm.head(3)

(1601, 13)


,id,from_id,date,text,post_id,owner_id,parents_stack,thread,attachments,is_from_post_author,deleted,reply_to_user,reply_to_comment
0,4546,45847070,1714983363,Кто автор?,4544.0,-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN
1,4541,260091507,1714430910,"""Всё страньше и страньше!"" ©Л. Кэрролл",4540.0,-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN
2,4537,2210113,1714247528,"Снимите, пожалуйста, пролаз круга по пассивам ...",4536.0,-38936316.0,[],"{'count': 1, 'items': [{'id': 4539, 'from_id':...",NaN,NaN,NaN,NaN,NaN


Обратите внимание: здесь id – это id комментария, не id поста, как раньше. Чтобы можно было совместить результаты, переименуем в `with_comm10` столбец `id` в `post_id`, чтобы не запутаться и понятным образом объединить датафреймы:

In [39]:
with_comm.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm.head(3)

,post_id,date,text,nlikes,ncomments,nviews,date_time
4,4544,1714901561,Воспитание психологической стойкости помогает ...,15,1,958,2024-05-05 09:32:41
6,4540,1714401519,,23,1,610,2024-04-29 14:38:39
7,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59


Объединим `with_comm10` и `df_comm` по столбцу `post_id` методом *LEFT JOIN*, к левому датафрейму `with_comm10` «подтянем» 
данные из правого `df_comm`:

In [40]:
final = with_comm.merge(df_comm, on = "post_id", how = "left")
final.head(5)

,post_id,date_x,text_x,nlikes,ncomments,nviews,date_time,id,from_id,date_y,text_y,owner_id,parents_stack,thread,attachments,is_from_post_author,deleted,reply_to_user,reply_to_comment
0,4544,1714901561,Воспитание психологической стойкости помогает ...,15,1,958,2024-05-05 09:32:41,4546,45847070,1714983363,Кто автор?,-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN
1,4540,1714401519,,23,1,610,2024-04-29 14:38:39,4541,260091507,1714430910,"""Всё страньше и страньше!"" ©Л. Кэрролл",-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN
2,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4537,2210113,1714247528,"Снимите, пожалуйста, пролаз круга по пассивам ...",-38936316.0,[],"{'count': 1, 'items': [{'id': 4539, 'from_id':...",NaN,NaN,NaN,NaN,NaN
3,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4538,260091507,1714263780,"Тем, кому интересно более глубоко и многоаспек...",-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN
4,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4527,260091507,1713775295,Это он ещё легко отделался! А то вот так доста...,-38936316.0,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN,NaN,NaN,NaN,NaN


Метод `.merge()` в случае одинаково названных столбцов с обоих датафреймах дописывает к названию из первого датафрейма `_x`, а к названию из второго – `_y`. Здесь `text_x` – это текст поста из первого датафрейма, а `text_y` – текст комментария из второго датафрейма, то же самое произошло с датами. Переименуем столбцы для ясности:

In [41]:
final.rename(columns = {"text_x" : "post_text", "text_y": "comment_text", 
                       "date_x" : "post_date", "date_y" : "comment_date",
                       "date_time" : "post_date_time"}, inplace = True)

Удалим лишние столбцы:

In [42]:
final.drop(columns = ["parents_stack", "attachments", "is_from_post_author", 
                      "deleted", "reply_to_user", "reply_to_comment"], inplace = True)
final.head()

,post_id,post_date,post_text,nlikes,ncomments,nviews,post_date_time,id,from_id,comment_date,comment_text,owner_id,thread
0,4544,1714901561,Воспитание психологической стойкости помогает ...,15,1,958,2024-05-05 09:32:41,4546,45847070,1714983363,Кто автор?,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
1,4540,1714401519,,23,1,610,2024-04-29 14:38:39,4541,260091507,1714430910,"""Всё страньше и страньше!"" ©Л. Кэрролл",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
2,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4537,2210113,1714247528,"Снимите, пожалуйста, пролаз круга по пассивам ...",-38936316.0,"{'count': 1, 'items': [{'id': 4539, 'from_id':..."
3,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4538,260091507,1714263780,"Тем, кому интересно более глубоко и многоаспек...",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
4,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4527,260091507,1713775295,Это он ещё легко отделался! А то вот так доста...,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."


Для полной ясности переименуем `id` в `comment_id` и `from_id` в `user_id`:

In [43]:
final.rename(columns = {"id" : "comment_id", "from_id" : "user_id"}, inplace = True)
final.head()

,post_id,post_date,post_text,nlikes,ncomments,nviews,post_date_time,comment_id,user_id,comment_date,comment_text,owner_id,thread
0,4544,1714901561,Воспитание психологической стойкости помогает ...,15,1,958,2024-05-05 09:32:41,4546,45847070,1714983363,Кто автор?,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
1,4540,1714401519,,23,1,610,2024-04-29 14:38:39,4541,260091507,1714430910,"""Всё страньше и страньше!"" ©Л. Кэрролл",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
2,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4537,2210113,1714247528,"Снимите, пожалуйста, пролаз круга по пассивам ...",-38936316.0,"{'count': 1, 'items': [{'id': 4539, 'from_id':..."
3,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4538,260091507,1714263780,"Тем, кому интересно более глубоко и многоаспек...",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."
4,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4527,260091507,1713775295,Это он ещё легко отделался! А то вот так доста...,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's..."


Выгрузим датафрейм в файл Excel на всякий случай, а сами перейдём к ещё одной бонусной части и выгрузим информацию о пользователях, оставивших комментарий. 

In [44]:
final.to_excel("rz_posts.xlsx")

## Выгружаем информацию по пользователям (бонус)

Предположим, мы хотим получить информацию о пользователях, которые оставили комментарии к постам. Так как одни и те люди могут оставлять комментарии к разным постам, выберем из столбца `user_id` только уникальные значения:

In [45]:
# id_uniq – массив numpy array

id_uniq = final["user_id"].unique()
print(len(id_uniq))

534


Метод `users.get` позволяет принимать на вход не один id пользователя, а сразу несколько, разделёнными запятыми. Объединим элементы в `id_uniq` в массивы одинаковой длины не более 10 (в последних массивах – сколько останется):

In [46]:
import numpy as np

# array_split() разобьет массив на n массивов одинакового размера
# если число элементов не кратно n, ошибки не будет 
# 534 // 10 + 1 – необходимое число массивов, чтобы было не больше 10 элементов

chunks = np.array_split(id_uniq, 534 // 10 + 1)

Посмотрим на первый и последним массивы:

In [47]:
print(chunks[0])
print(chunks[-1])

[ 45847070 260091507   2210113 784213526  17426424   8437598 510158360
  41414377  19182024   4307965]
[137607337 174389709    162291   1942878   1037222   3058978  34410432
   4203900  57556726]


Проверим длину массивов:

In [48]:
# всего 6 массивов с одинаковым числом элементов

print(len(chunks), list(map(len, chunks)))

54 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9]


Склеиваем элементы внутри каждого массива в строку (перед этим превращаем массивы в массивы строк):

In [49]:
id_joined = [",".join(ch.astype(str)) for ch in chunks]

Проверяем:

In [50]:
print(id_joined[0:2])

['45847070,260091507,2210113,784213526,17426424,8437598,510158360,41414377,19182024,4307965', '95826274,240112961,840496946,13143639,196636514,1191246,724224,624671,146913425,113333128']


Пробуем подавать такие строки на вход методу `users.get` с некоторой задержкой:

In [54]:
main_users = "https://api.vk.com/method/users.get"

# немного другой подход: чтобы не создавать новый словарь каждый раз,
# будем добавлять в него только новую запись про id

params_users = {"access_token" : token, "v" : v}

users_all = []

for i_str in id_joined:
    params_users["user_ids"] = i_str 
    req = requests.get(main_users, params = params_users)
    users_json = req.json()
    users_list = users_json["response"]
    users_all.extend(users_list)
    time.sleep(1)

In [55]:
df_users = pd.DataFrame(users_all)

In [56]:
df_users.head()

,id,first_name,last_name,can_access_closed,is_closed,deactivated
0,45847070,Анастасия,Артюхова,True,False,NaN
1,260091507,Владимир,Цепаев,True,False,NaN
2,2210113,Владимир,Хабров,True,False,NaN
3,784213526,Лиза,Винокурова,True,False,NaN
4,17426424,Елена,Петрова,True,False,NaN


Оставим в `df_users` только id, имя и фамилию:

In [57]:
df_users = df_users.loc[:, "id" : "last_name"]

Объединим `final` и `df_users` по id пользователя, сообщив, что `user_id` в первом датафрейме – это то же самое, что `id` во втором:

In [60]:
finfin = final.merge(df_users, left_on = "user_id", right_on = "id", how = "left")

In [61]:
finfin.head(10)

,post_id,post_date,post_text,nlikes,ncomments,nviews,post_date_time,comment_id,user_id,comment_date,comment_text,owner_id,thread,id,first_name,last_name
0,4544,1714901561,Воспитание психологической стойкости помогает ...,15,1,958,2024-05-05 09:32:41,4546,45847070,1714983363,Кто автор?,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",45847070.0,Анастасия,Артюхова
1,4540,1714401519,,23,1,610,2024-04-29 14:38:39,4541,260091507,1714430910,"""Всё страньше и страньше!"" ©Л. Кэрролл",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",260091507.0,Владимир,Цепаев
2,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4537,2210113,1714247528,"Снимите, пожалуйста, пролаз круга по пассивам ...",-38936316.0,"{'count': 1, 'items': [{'id': 4539, 'from_id':...",2210113.0,Владимир,Хабров
3,4536,1714234979,"Тренировки на выносливость - это, пожалуй, оди...",16,3,882,2024-04-27 16:22:59,4538,260091507,1714263780,"Тем, кому интересно более глубоко и многоаспек...",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",260091507.0,Владимир,Цепаев
4,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4527,260091507,1713775295,Это он ещё легко отделался! А то вот так доста...,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",260091507.0,Владимир,Цепаев
5,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4528,784213526,1713783031,Так и попали все на скалодром🙃,-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",784213526.0,Лиза,Винокурова
6,4526,1713722295,,51,3,843,2024-04-21 17:58:15,4531,17426424,1713812684,"Легко отделался, придя на скалодром) вот если ...",-38936316.0,"{'count': 0, 'items': [], 'can_post': True, 's...",17426424.0,Елена,Петрова
7,4518,1713429306,17 апреля у нас проходил мастер-класс по лазан...,11,4,849,2024-04-18 08:35:06,4519,8437598,1713435932,Мастер-класс по скалолазанию провели великолеп...,-38936316.0,"{'count': 1, 'items': [{'id': 4520, 'from_id':...",8437598.0,Анастасия,Лова
8,4518,1713429306,17 апреля у нас проходил мастер-класс по лазан...,11,4,849,2024-04-18 08:35:06,4522,260091507,1713454515,"Ага, стало быть, были все, кроме меня... 😒",-38936316.0,"{'count': 1, 'items': [{'id': 4524, 'from_id':...",260091507.0,Владимир,Цепаев
9,4509,1713199527,"В субботу, 13 апреля на нашем скалодроме состо...",21,3,1144,2024-04-15 16:45:27,4510,510158360,1713208313,Не записывали?,-38936316.0,"{'count': 2, 'items': [{'id': 4511, 'from_id':...",510158360.0,Григорий,Кулага


Ура! Выгружаем в файл:

In [62]:
finfin.to_excel("posts_with_users.xlsx")